In [1]:
import os
import sys

# Get the current working directory (where the notebook is located)
notebook_dir = os.getcwd()

# Get the parent directory
parent_dir = os.path.abspath(os.path.join(notebook_dir, os.pardir))

# Set the parent directory as the working directory
os.chdir(parent_dir)

# Optionally, you can add the parent directory to the system path
sys.path.append(parent_dir)

# Print the current working directory to verify
print("Current working directory:", os.getcwd())

Current working directory: /ictstr01/home/icb/fatemehs.hashemig/codes/interpretable-ssl


In [2]:
from experiment_runner import *

In [3]:
print(1)

1


In [5]:
item_to_test = {
            "dimensionality_reduction": ["pca", None],
            "prot_decoding_loss_scaler": [0, 5],
            "cvae_loss_scaler": [0, 0.0001],
            "k_neighbors": [5, 10],
            "semi_supervised": [True],
            "augmentation_type": ["scanpy_knn", "community", 'knn'],
            
        }
        


In [6]:
keys, values = zip(*item_to_test.items())

In [8]:
value_combination = next(iter(itertools.product(*values)))

In [9]:
self = ExperimentRunner("swav_template.sbatch")

In [10]:
params = dict(zip(keys, value_combination))

# Generate job name automatically based on differences from defaults
job_name = self.generate_job_name(params)
job_name

'aug_scan_DR_pca_kN_5_semi_True'

In [14]:
def get_slurm_job_names():
    user = 'fatemehs.hashemig'
    try:
        # Run the 'squeue' command for the specified user and capture the output
        result = subprocess.run(['squeue', '--user', user, '--format', '%.100j'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

        # Check for errors
        if result.returncode != 0:
            print(f"Error: {result.stderr}")
            return []

        # Split the output into lines, strip leading and trailing whitespaces
        lines = [line.strip() for line in result.stdout.splitlines()]

        # Skip the header (first line) and return the remaining lines (job names)
        return lines[1:]  # Skip the first line if it contains the header

    except Exception as e:
        print(f"An error occurred: {e}")
        return []

In [15]:
current_jobs = get_slurm_job_names()
current_jobs

['aug_knn_kN_5_semi_True',
 'aug_comm_kN_5_semi_True',
 'aug_scan_kN_5_semi_True',
 'JPTR',
 'aug_scan_DR_pca_kN_5_semi_True',
 'aug_knn_DR_pca_kN_5_semi_True',
 'aug_knn_DR_pca_semi_True',
 'aug_knn_DR_pca_cvaeLS_0.0001_kN_5_semi_True',
 'aug_scan_DR_pca_kN_5_semi_True',
 'aug_comm_DR_pca_PDLS_5_kN_5_semi_True',
 'aug_scan_DR_pca_PDLS_5_kN_5_semi_True',
 'aug_comm_DR_pca_PDLS_5_semi_True',
 'aug_scan_DR_pca_PDLS_5_semi_True',
 'aug_knn_DR_pca_PDLS_5_kN_5_semi_True',
 'aug_knn_DR_pca_PDLS_5_semi_True']

In [16]:
job_name in current_jobs

True

In [18]:
self.update_params(**params)

In [20]:
sbatch_content = self.generate_sbatch_content()


In [21]:
sbatch_content

'#!/bin/bash\n#SBATCH --job-name=default_job\n#SBATCH --nodes=1\n#SBATCH --cpus-per-task=20\n#SBATCH --mem=128G\n#SBATCH --output=slurm-job-out/default_job/output.txt\n#SBATCH --error=slurm-job-out/default_job/error.txt\n#SBATCH --nice=10000\n#SBATCH --partition=gpu_p\n#SBATCH --qos=gpu_short\n#SBATCH --gres=gpu:1\n\nsource ~/.bashrc\nconda activate apex-env\n\npython main.py swav --num_prototypes 300 --latent_dims 8 --batch_size 1024 \\\n--augmentation_type scanpy_knn --epsilon 0.02 --cvae_loss_scaler 0 \\\n--prot_decoding_loss_scaler 0 --model_version 1 \\\n--dimensionality_reduction pca --k_neighbors 5 --experiment_name None\n'